In [1]:
# Open the text file and count the lines
with open('/home/jupyter-23521027/refresh-bert/data/indo4b/dataset_all_uncased_blankline.txt', 'r') as file:
    line_count = sum(1 for line in file)

# Print the number of lines
print("Number of lines:", line_count)

Number of lines: 373648582


In [31]:
import nltk
from nltk.tokenize import word_tokenize
from datetime import datetime

# Preprocessing step: tokenization and cleaning

# Define the context window size
window_size = 2

# Open the text file
with open('/home/jupyter-23521027/refresh-bert/data/indo4b/dataset_all_uncased_blankline.txt', 'r') as file:
    # Iterate over each line in the file
    now = datetime.now()
    for l, line in enumerate(file):
        # Tokenize the line into words
        words = word_tokenize(line)
        
        # Generate word pairs
        word_pairs = []
        for i, target_word in enumerate(words):
            for j in range(i - window_size, i + window_size + 1):
                if j != i and j >= 0 and j < len(words):
                    context_word = words[j]
                    word_pairs.append((target_word, context_word))
                    
        if ( l + 1 ) % 100000 == 0:
            print(f"processed {l+1} files in : {datetime.now() - now}")
            now = datetime.now()
        
#         if l == 500000:
#             break

processed 100000 files in : 0:00:11.600052
processed 200000 files in : 0:00:11.579006
processed 300000 files in : 0:00:11.571496
processed 400000 files in : 0:00:11.530318
processed 500000 files in : 0:00:11.418972


In [33]:
word_pairs[:2]

[('gunakan', 'waktu'), ('gunakan', 'anda')]

In [34]:
# Assuming you have word_pairs as a list of word pairs

# Create word-to-index and index-to-word mappings
words = set([pair[0] for pair in word_pairs] + [pair[1] for pair in word_pairs])
print(words)
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

# Convert word pairs to numerical representations
numeric_pairs = [(word_to_index[pair[0]], word_to_index[pair[1]]) for pair in word_pairs]

{'amati', 'sebelum', 'dalam', '.', 'dengan', 'bagian', 'setiap', 'gunakan', 'proses', 'pemeriksaan', 'ini', 'dan', 'teliti', 'menyelesaikan', 'waktu', 'anda', 'baik', 'pendaftaran'}


In [35]:
word_to_index

{'amati': 0,
 'sebelum': 1,
 'dalam': 2,
 '.': 3,
 'dengan': 4,
 'bagian': 5,
 'setiap': 6,
 'gunakan': 7,
 'proses': 8,
 'pemeriksaan': 9,
 'ini': 10,
 'dan': 11,
 'teliti': 12,
 'menyelesaikan': 13,
 'waktu': 14,
 'anda': 15,
 'baik': 16,
 'pendaftaran': 17}

In [55]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Activation

# Convert numeric pairs to numpy arrays
target_words = np.array([pair[0] for pair in numeric_pairs])
context_words = np.array([pair[1] for pair in numeric_pairs])

In [73]:
# Build the skip-gram model
vocab_size = len(word_to_index)
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, embeddings_initializer="glorot_uniform",))
model.add(Dense(units=1000, activation='tanh'))
model.add(Dense(units=1000, activation='tanh'))
model.add(Dense(units=1000, activation='tanh'))
model.add(Dense(units=vocab_size, activation='softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [74]:
# Train the model
model.fit(target_words, context_words, epochs=10, batch_size=128)


Epoch 1/10
1/1 [==============================] - 1s 732ms/step - loss: 2.9011
Epoch 2/10
1/1 [==============================] - 0s 9ms/step - loss: 2.0188
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 1.8503
Epoch 4/10
1/1 [==============================] - 0s 13ms/step - loss: 2.3888
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 2.2429
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 1.9531
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 1.8043
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 1.7905
Epoch 9/10
1/1 [==============================] - 0s 11ms/step - loss: 1.8702
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 1.8406


In [58]:
print(model.summary())


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1, 100)            1800      
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dense_12 (Dense)            (None, 100)               10100     
                                                                 
 dense_13 (Dense)            (None, 18)                1818      
                                                                 
Total params: 23,818
Trainable params: 23,818
Non-trainable params: 0
_________________________________________________________________
None
